##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [2]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [3]:
saut="\n"

###Préparation des matrices de traits

In [4]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [5]:
samplePrefix="MGC-150812-30000-1508120226-X-Regles.pkl"
sampleFile=samplePrefix+".csv"
goldFile="MGC-150815-total-1508150555-paradigmes.csv"
analysisPrefix="MGC-150812-3000-1508140124-S"
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open(logfile_name,mode='w',encoding="utf8")

###Préparation des cases du paradigme

In [6]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

#Préparation du calcul des analogies

###Calcul de la différence entre deux formes

In [7]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

###Accumulation des paires appartenant à un patron

In [8]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

###Transformation d'un patron en RegExp

In [9]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

###Substitution de sortie 
???

In [10]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

#Classe pour la gestion des patrons, des classes et des transformations

In [11]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [12]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [13]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [14]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [15]:
paradigmes=pd.read_csv(sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]

In [16]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [17]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [18]:
GOLD=pd.read_csv(goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
goldCases=GOLD.columns.tolist()
goldCases.remove(u"lexeme")
for case in goldCases:
    GOLD[case]=GOLD[case].apply(lambda x: recoder(x))
    
paradigmesGOLD=GOLD[paradigmes.columns.values.tolist()]
paradigmesGOLD

,lexeme,ai1S,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,fi2S,fi3P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,abEsE,abEsa,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6rE,abEs6ra,abEs6rô,...,abEsE,abEsE,abEsE,abEsE,abEsjô,abEs,abEsjE,abEs,abEs,abEs
1,abandonner,abâdOnE,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6rE,abâdOn6ra,abâdOn6rô,...,abâdOnE,abâdOnE,abâdOnE,abâdOnE,abâdOnjô,abâdOn,abâdOnjE,abâdOn,abâdOn,abâdOn
2,abasourdir,abazurdi,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdirE,abazurdira,abazurdirô,...,abazurdi,abazurdi,abazurdi,abazurdi,abazurdisjô,abazurdis,abazurdisjE,abazurdis,abazurdis,abazurdis
3,abattre,abati,abati,abatir,abati,abatrô,abatrE,abatrE,abatra,abatrô,...,abaty,abaty,abaty,abaty,abatjô,abat,abatjE,abat,abat,abat
4,abcéder,absEdE,absEda,absEdEr,absEda,absEd6rô,absEd6rE,absEd6rE,absEd6ra,absEd6rô,...,NaN,NaN,NaN,absEdE,absEdjô,absEd,absEdjE,absEd,absEd,absEd
5,abdiquer,abdikE,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6rE,abdik6ra,abdik6rô,...,abdikE,abdikE,abdikE,abdikE,abdikjô,abdik,abdikjE,abdik,abdik,abdik
6,abecquer,abEkE,abEka,abEkEr,abEka,abEk6rô,abEk6rE,abEk6rE,abEk6ra,abEk6rô,...,abEkE,abEkE,abEkE,abEkE,abEkjô,abEk,abEkjE,abEk,abEk,abEk
7,aberrer,abErE,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6rE,abEr6ra,abEr6rô,...,NaN,NaN,NaN,abErE,abErjô,abEr,abErjE,abEr,abEr,abEr
8,abhorrer,abOrE,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6rE,abOr6ra,abOr6rô,...,abOrE,abOrE,abOrE,abOrE,abOrjô,abOr,abOrjE,abOr,abOr,abOr
9,abjurer,abZyrE,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6rE,abZyr6ra,abZyr6rô,...,abZyrE,abZyrE,abZyrE,abZyrE,abZyrjô,abZyr,abZyrjE,abZyr,abZyr,abZyr


- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [19]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [20]:
paradigmes.stack().value_counts(dropna=True).sum()

4000

In [21]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

###Comparer les cases analysées avec l'ensemble de toutes les cases

In [22]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(casesTotales):
    print ("Attention l'analyse ne comprend pas toutes les cases")

Attention l'analyse ne comprend pas toutes les cases


In [23]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [24]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [25]:
paradigmes[paradigmes["lexeme"]=="dormir"].columns[paradigmes[paradigmes["lexeme"]=="dormir"].notnull().iloc[0]].tolist()

[u'lexeme',
 u'ai3S',
 u'ii3P',
 u'inf',
 u'pI2P',
 u'pi1S',
 u'pi2S',
 u'pi3S',
 u'ps3P']

In [26]:
def ajouterPoint(lexeme,forme,case):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [27]:
def generateParadigms(generation1):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortie=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortie:
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie))
    return candidats

In [28]:
def generate(lexeme):
    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    generation2=generateParadigms(generation1)
#    print ("génération 3")
    return generation2

In [29]:
len(paradigmes.dropna(thresh=6)["lexeme"])

184

In [30]:
%%time
debug=False
listeTest=[u"manger",u"boire",u"dormir",u"aller",u"avoir"]
#listeTest=paradigmes.dropna(thresh=3)["lexeme"].values.tolist()
digraphe=nx.DiGraph()
graphe=nx.Graph()
for i,element in enumerate(listeTest):
    print (i,end=", ")
    generate(element)

0, manger, 1, boire, 2, dormir, 3, aller, 4, avoir, CPU times: user 3.41 s, sys: 55.9 ms, total: 3.46 s
Wall time: 3.46 s


In [31]:
%%time
versionStamp=dateheure()
nx.write_dot(digraphe,u"digraphe-%s.dot"%versionStamp)
nx.write_dot(graphe,u"graphe-%s.dot"%versionStamp)

CPU times: user 392 ms, sys: 50.3 ms, total: 442 ms
Wall time: 427 ms


In [32]:
%%time
cliques=list(nx.algorithms.clique.find_cliques(graphe))

CPU times: user 10.4 ms, sys: 2.59 ms, total: 13 ms
Wall time: 10.9 ms


In [33]:
for clique in sorted(cliques,key=lambda x: len(x),reverse=True)[:50]:
    print (len(clique),end=", ")
    print (clique)

25, [u'avoir-av\xf4-pi1P', u'avoir-avE-ii2S', u'avoir-avE-ii3S', u'avoir-avE-ii1S', u'avoir-avE-pi2P', u'avoir-avE-ii3P', u'avoir-a-pi2S', u'avoir-E-pi1S', u'avoir-a-pi3S', u'avoir-y-ppMS', u'avoir-EjE-pI2P', u'avoir-avwar-inf', u'avoir-Or\xf4-fi3P', u'avoir-EjE-ps2P', u'avoir-y-ppMP', u'avoir-E-pI2S', u'avoir-avj\xf4-ii1P', u'avoir-Ora-fi3S', u'avoir-OrE-fi2P', u'avoir-OrE-pc1S', u'avoir-Or\xf4-fi1P', u'avoir-OrE-fi1S', u'avoir-Ora-fi2S', u'avoir-y-ai3S', u'avoir-\xf4-pi3P']
23, [u'aller-vE-pi1S', u'aller-alE-ii2S', u'aller-alE-ii3S', u'aller-alE-pi2P', u'aller-alE-ppFS', u'aller-alE-inf', u'aller-irE-pc3S', u'aller-alE-pI2P', u'aller-ira-fi3S', u'aller-alE-ppMS', u'aller-al\xf4-pI1P', u'aller-irj\xf4-pc1P', u'aller-v\xf4-pi3P', u'aller-irE-fi2P', u'aller-ala-ai3S', u'aller-aj-ps1S', u'aller-aj-ps3S', u'aller-aljE-ii2P', u'aller-alE-ii1S', u'aller-va-pi3S', u'aller-va-pI2S', u'aller-va-pi2S', u'aller-al\xf4-pi1P']
14, [u'boire-byvE-ii2S', u'boire-byv-ps1S', u'boire-byv-ps3S', u'boire-

In [34]:
nombreElements={element:paradigmes[paradigmes["lexeme"]==element].stack().value_counts(dropna=True).sum()-1 for element in listeTest}
for element in sorted(nombreElements,key=nombreElements.get, reverse=True):
    print (element, nombreElements[element])

avoir 25
aller 23
dormir 8
manger 6
boire 4


##Comparer la sortie des cliques avec le paradigme de départ

In [39]:
%%time
seuilClique=3
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        lexeme=clique[0].split("-")[0]
        paradigmeClique["lexeme"]=lexeme
        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
        casesLexeme.remove("lexeme")
        for element in casesLexeme:
            pointCase=u"%s-%s-%s"% (lexeme,paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0],element)
            if debug: print (pointCase, clique)
            if not pointCase in clique:
                sampleOK=False        
        if sampleOK:
            for element in clique:
                (lexeme,forme,taminfo)=element.split("-")
                paradigmeClique[taminfo]=forme
                if taminfo in paradigmes.columns:
                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
                            sampleOK=False
                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist())
print (seuilClique)

3
CPU times: user 1.41 s, sys: 12.9 ms, total: 1.43 s
Wall time: 1.44 s


In [40]:
paradigmesCLIQUES

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,avoir,NaN,NaN,NaN,NaN,NaN,y,Orô,OrE,OrE,...,NaN,NaN,y,y,NaN,NaN,EjE,NaN,NaN,NaN
1,aller,NaN,NaN,NaN,NaN,NaN,ala,NaN,NaN,irE,...,NaN,alE,NaN,alE,NaN,aj,NaN,NaN,NaN,aj
2,dormir,NaN,NaN,NaN,NaN,NaN,dOrmi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dOrm,NaN
3,manger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mâZ6rE,NaN,...,NaN,NaN,NaN,mâZE,NaN,NaN,NaN,NaN,NaN,NaN
4,boire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,by,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#paradigmesCLIQUES
for element in listeTest:
    if paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==element].stack().value_counts(dropna=True).sum()>1:
        print (element,paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==element].stack().value_counts(dropna=True).sum()-1)


manger 6
boire 4
dormir 8
aller 23
avoir 25


In [42]:
GOLD.loc[GOLD["lexeme"]=="dormir"].stack()

1545  lexeme      dormir
      ai1P        dOrmim
      ai1S         dOrmi
      ai2P        dOrmit
      ai2S         dOrmi
      ai3P        dOrmir
      ai3S         dOrmi
      fi1P       dOrmirô
      fi1S       dOrmirE
      fi2P       dOrmirE
      fi2S       dOrmira
      fi3P       dOrmirô
      fi3S       dOrmira
      ii1P        dOrmjô
      ii1S         dOrmE
      ii2P        dOrmjE
      ii2S         dOrmE
      ii3P         dOrmE
      ii3S         dOrmE
      inf         dOrmir
      is1P      dOrmisjô
      is1S        dOrmis
      is2P      dOrmisjE
      is2S        dOrmis
      is3P        dOrmis
      is3S         dOrmi
      pI1P         dOrmô
      pI2P         dOrmE
      pI2S           dOr
      pP           dOrmâ
      pc1P      dOrmirjô
      pc1S       dOrmirE
      pc2P      dOrmirjE
      pc2S       dOrmirE
      pc3P       dOrmirE
      pc3S       dOrmirE
      pi1P         dOrmô
      pi1S           dOr
      pi2P         dOrmE
      pi2S           dOr


In [53]:
def compareGOLD(row):
#    if row["lexeme"]=="dodo": 
#        debug=True
#    else:
#        debug=False
    print (row["lexeme"],end=" : ")
    lexeme=row["lexeme"]
    if not lexeme in conjugaisons:
        conjugaisons[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    identiques=[]
    differents=[]
    missing=[]
    over=[]
    for case in analyseCases:
        if debug:    
            print (lexeme,case)
            print (GOLD[GOLD["lexeme"]==lexeme][case].item())
            print ((GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]))
        if not case in row:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("absence",case)
        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            over.append(u"%s : %s ≠ Ø" % (case,row[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
        elif (row[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
            if debug: print ("egal")
            identiques.append(u"%s : %s" % (case,row[case]))
            if debug: print ("valeurs id")
        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
            if debug: print ("NaNs")
            identiques.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
            differents.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if differents:
        conjugaisons[lexeme]["diff"].append(differents)
        if debug: print ("DIFF",len(differents),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: print (", ".join(differents))
    if missing and over:
        conjugaisons[lexeme]["overmiss"].append(over+missing)
        if debug: print ("OVERMISS", len(over), len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
            print (", ".join(over))
    elif missing:
        conjugaisons[lexeme]["miss"].append(missing)
        if debug: print ("MISS", len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
    elif over:
        conjugaisons[lexeme]["over"].append(over)
        if debug: print ("OVER", len(over),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(over))
    if identiques:
        conjugaisons[lexeme]["ok"].append(identiques)
        if debug: print ("OK", len(identiques), len(identiques)-nombreElements[lexeme],end=saut)
        if debug1: print (", ".join(identiques))
#    print ()
    return lexical

In [54]:
%%time
debug=False
debug1=False
saut="\n"
conjugaisons={}
print (", ".join(sampleCases))
paradigmesCLIQUES.apply(compareGOLD,axis=1)

ai1S, ai2S, ai3P, ai3S, fi1P, fi1S, fi2P, fi2S, fi3P, fi3S, ii1P, ii1S, ii2P, ii2S, ii3P, ii3S, inf, is1S, is3P, is3S, pI1P, pI2P, pI2S, pP, pc1P, pc1S, pc2P, pc2S, pc3P, pc3S, pi1P, pi1S, pi2P, pi2S, pi3P, pi3S, ppFP, ppFS, ppMP, ppMS, ps1P, ps1S, ps2P, ps2S, ps3P, ps3S
avoir : aller : dormir : manger : boire : CPU times: user 716 ms, sys: 8.04 ms, total: 724 ms
Wall time: 730 ms


In [55]:
pb=[i for i in conjugaisons if conjugaisons[i]["diff"]]
for element in pb:
    print (element)
    print ("DIFF",conjugaisons[element]["diff"])
    print ("MISS",conjugaisons[element]["miss"],end="\n\n")

In [68]:
good=[i for i in conjugaisons if not conjugaisons[i]["diff"]]
for lexeme in good:
    print (lexeme)
    for ident in conjugaisons[lexeme]:
        for element in conjugaisons[lexeme][ident]:
            print (ident)
            print (", ".join(element))
    print ()

manger
miss
ii1P : Ø ≠ mâZjô, pP : Ø ≠ mâZâ, pc1P : Ø ≠ mâZ6rjô, ii1S : Ø ≠ mâZE, ai3S : Ø ≠ mâZa, ppMP : Ø ≠ mâZE, is3P : Ø ≠ mâZas, is3S : Ø ≠ mâZa, ii3S : Ø ≠ mâZE, ps3S : Ø ≠ mâZ, ii3P : Ø ≠ mâZE, pi2S : Ø ≠ mâZ, ai1S : Ø ≠ mâZE, pi2P : Ø ≠ mâZE, ppFS : Ø ≠ mâZE, ppFP : Ø ≠ mâZE, ps1P : Ø ≠ mâZjô, ps3P : Ø ≠ mâZ, ps1S : Ø ≠ mâZ, fi2P : Ø ≠ mâZ6rE, fi2S : Ø ≠ mâZ6ra, ps2P : Ø ≠ mâZjE, ps2S : Ø ≠ mâZ, ai2S : Ø ≠ mâZa, pc2P : Ø ≠ mâZ6rjE, pc2S : Ø ≠ mâZ6rE, ii2P : Ø ≠ mâZjE, pi1S : Ø ≠ mâZ, pi1P : Ø ≠ mâZô, ii2S : Ø ≠ mâZE, pi3S : Ø ≠ mâZ, pc3S : Ø ≠ mâZ6rE, pc3P : Ø ≠ mâZ6rE, is1S : Ø ≠ mâZas, pc1S : Ø ≠ mâZ6rE, fi3S : Ø ≠ mâZ6ra, fi3P : Ø ≠ mâZ6rô, ai3P : Ø ≠ mâZEr, fi1P : Ø ≠ mâZ6rô, pI1P : Ø ≠ mâZô
ok
inf : mâZE, pI2S : mâZ, pI2P : mâZE, pi3P : mâZ, ppMS : mâZE, fi1S : mâZ6rE

boire
miss
ii1P : Ø ≠ byvjô, pP : Ø ≠ byvâ, pc1P : Ø ≠ bwarjô, ii1S : Ø ≠ byvE, ai3S : Ø ≠ by, ppMP : Ø ≠ by, is3P : Ø ≠ bys, is3S : Ø ≠ by, ii3S : Ø ≠ byvE, ps3S : Ø ≠ bwav, ii3P : Ø ≠ byvE, pi2S : Ø ≠ bwa,

In [ ]:
paradigmes[paradigmes["lexeme"]==u"éprouver"].stack()

In [ ]:
GOLD[GOLD["lexeme"]==u"éprouver"]

In [ ]:
paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==u"éprouver"].stack()

In [ ]:
paradigmesGOLD

In [ ]:
paradigmesGOLD.loc[paradigmesGOLD["lexeme"]=="boire"]["inf"]